# Import libraries

In [ ]:
import IPython.display
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# you have to install ipython-autotime using 'pip install ipython-autotime'
%load_ext autotime


# in this project, the metric is rmse, not mse
from sklearn.metrics import mean_squared_error

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.neighbors import KNeighborsRegressor

# Load datasets

In [ ]:
sales = pd.read_csv('./dataset/sales_train.csv.gz')
shops = pd.read_csv('./dataset/shops.csv')
items = pd.read_csv('./dataset/items.csv')
item_cats = pd.read_csv('./dataset/item_categories.csv')

# Make utilities to submit

In [ ]:
def make_submission_df(all_prediction):
    test = pd.read_csv("./dataset/test.csv.gz")
    df = test.merge(all_prediction, on=["shop_id", "item_id"], how="left")[["ID", "item_cnt_month"]]
    
    # clip the predicted values between 0 and 20
    df["item_cnt_month"] = df["item_cnt_month"].fillna(0).clip(0, 20)
    
    return df

def make_submission_file(df, comment="", add_time_stamp=True):
    name = "submission"
    
    if add_time_stamp:
        name = "%s_%d" % (name, time.time())
    
    if len(comment) > 0:
        name = "%s_%s" % (name, comment)
        
    df.to_csv("./submission/%s.csv" % name, sep=",", index=False)
    
def make_submission(all_prediction, comment="", add_time_stamp=True):
    make_submission_file(make_submission_df(all_prediction), comment, add_time_stamp)

# Make benchmarks

There should be benchmarks to measure my prediction's quality, so I made very simple ones.

In [ ]:
sample = pd.read_csv('./dataset/sample_submission.csv.gz')
make_submission_file(sample, 'sample_value', False)

sample['item_cnt_month'] = 0
make_submission_file(sample, 'zero_value', False)

previous_month = sales[sales["date_block_num"] == 33].groupby(["shop_id", "item_id"], as_index=False).item_cnt_day.sum().rename(columns={"item_cnt_day": "item_cnt_month"})
make_submission(previous_month, "previous_month_value", False)

* sample value: 1.23646
* zero value: 1.25011
* previous month value: 1.16777

In [ ]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# Get a feature matrix

In [ ]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

from itertools import product
# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)

del grid, gb 

import gc
gc.collect();

After creating a grid, we can calculate some features. We will use lags from [1, 2, 3, 4, 5, 12] months ago.

In [ ]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

from tqdm import tqdm_notebook

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013(because we use 12 months lag data in the target)
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

To this end, we've created a feature matrix. It is stored in `all_data` variable. Take a look:

In [ ]:
all_data

# Train/test split

In [ ]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

In [ ]:
%%time
dates_train = dates[dates <  last_block]
dates_valid  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_valid =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_valid =  all_data.loc[dates == last_block, 'target'].values

# First level models 

In [ ]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_valid.values)


rmse = np.sqrt(mean_squared_error(y_valid, pred_lr))
print('Test rmse for linreg is %f' % rmse)

rmse = np.sqrt(mean_squared_error(np.clip(y_valid, 0, 20), np.clip(pred_lr, 0, 20)))
print('Clipped Test rmse for linreg is %f' % rmse)

In [ ]:
rf = RandomForestRegressor(max_depth=5, random_state=0)
rf.fit(X_train.values, y_train)
pred_rf = rf.predict(X_valid.values)

rmse =  np.sqrt(mean_squared_error(y_valid, pred_rf))
print('Test rmse for RandomForest is %f' % rmse)

rmse = np.sqrt(mean_squared_error(np.clip(y_valid, 0, 20), np.clip(pred_rf, 0, 20)))
print('Clipped Test rmse for RandomForest is %f' % rmse)

# Submit to kaggle

This cell automatically submits the submission file to kaggle and shows all the results of submissions. However, it should be carefully executed because the submitting opportunities are limited.

In [ ]:
#!kaggle competitions submit -c competitive-data-science-final-project -f ./submission/submission_sample_value.csv -m "description"
!kaggle competitions submissions -c competitive-data-science-final-project